

# NMT using attention
## NOTICE : Attention layer is not yet implemented in keras library , which makes implementation of attention mechanism a lot difficult than other neural network models
## We can't use keras's model class and do model.fit , instead we'll have to write optimizer, loss function, model layers and training code from scratch and combine them to create a workflow. This is equivalent to writing model.fit in normal neural network models.

In [3]:

import tensorflow as tf
tf.enable_eager_execution()
from __future__ import absolute_import, division, print_function, unicode_literals


import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from sklearn.model_selection import train_test_split

import unicodedata
import re
import numpy as np
import os
import io
import time
import string

# loading the 1st dataset as it contains more smaller sentences which might help in better learning


In [4]:
# download dataset
!wget 'http://www.manythings.org/anki/hin-eng.zip'
!unzip 'hin-eng.zip'
lines = io.open('hin.txt', encoding='UTF-8').read().strip().split('\n')
path_to_file = 'hin.txt'

eng_sent = []
hin_sent = []
for line in lines:
  splits = line.split('\t')
  #removing extra stuffs 
  eng = splits[0]
  hin = splits[1]
  # remove punctutation and make lower case
  eng = eng.translate(str.maketrans('', '', string.punctuation))
  eng = eng.lower()
  hin = hin.translate(str.maketrans('', '', string.punctuation))
  eng = '<start> ' + eng + ' <end>'
  hin = '<start> ' + hin + ' <end>'
  eng_sent.append(eng)
  hin_sent.append(hin)

--2020-03-19 20:34:10--  http://www.manythings.org/anki/hin-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.108.196, 104.24.109.196, 2606:4700:3037::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.108.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 126449 (123K) [application/zip]
Saving to: ‘hin-eng.zip.1’

hin-eng.zip.1       100%[===================>] 123.49K   711KB/s    in 0.2s    

2020-03-19 20:34:10 (711 KB/s) - ‘hin-eng.zip.1’ saved [126449/126449]

Archive:  hin-eng.zip
replace hin.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: hin.txt                 
  inflating: _about.txt              


## Download and process 2nd dataset
## this has 7 lakhs translations of which we'll use 90k whose length are less than 50 words because of system's limitation.

In [5]:
# download dataset
!wget 'http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/prunedCorpus.tar.gz'


--2020-03-19 20:34:49--  http://www.cfilt.iitb.ac.in/~moses/iitb_en_hi_parallel/iitb_corpus_download/prunedCorpus.tar.gz
Resolving www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)... 103.21.127.130
Connecting to www.cfilt.iitb.ac.in (www.cfilt.iitb.ac.in)|103.21.127.130|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84265584 (80M) [application/x-gzip]
Saving to: ‘prunedCorpus.tar.gz’

prunedCorpus.tar.gz 100%[===================>]  80.36M  5.01MB/s    in 32s     

2020-03-19 20:35:21 (2.55 MB/s) - ‘prunedCorpus.tar.gz’ saved [84265584/84265584]



In [6]:
!ls
# !unzip 'prunedCorpus.tar.gz'
!tar -xvzf 'prunedCorpus.tar.gz'


_about.txt   hin-eng.zip.1  prunedCorpus.tar.gz
hin-eng.zip  hin.txt	    sample_data
pruned_train.en
pruned_train.hi


In [7]:
!ls

_about.txt   hin-eng.zip.1  prunedCorpus.tar.gz  pruned_train.hi
hin-eng.zip  hin.txt	    pruned_train.en	 sample_data


In [0]:
lines_hin = io.open('pruned_train.hi', encoding='UTF-8').read().strip().split('\n')
path_to_file = 'pruned_train.hi'

lines_eng = io.open('pruned_train.en', encoding='UTF-8').read().strip().split('\n')
path_to_file = 'pruned_train.en'

In [9]:
for i in range(5):
  print(lines_eng[i],lines_hin[i])

We generally find this capacity being directed towards mundane objects and close relations like wives , children and friends .  प्रायः हम इस भंडार को लौकिक विषयों पर और पत्नी , पुत्र मित्र आदि सगे संबंधियों पर प्रवर्तित कर देते हैं । 
I ' m sure I ' ll make a careless mistake मुझे जनता हूँ की मैं इस वीडियो में किसी भी समय
Display web browser help वेब ब्राउज़र मदद दिखाएँ
It is out of His Mercy that He has made for you Night and Day , - that ye may rest therein , and that ye may seek of his Grace ; - and in order that ye may be grateful .  और उसने अपनी मेहरबानी से तुम्हारे वास्ते रात और दिन को बनाया ताकि तुम रात में आराम करो और दिन में उसके फज़ल व करम की तलाश करो और ताकि तुम लोग शुक्र करो
The disease caused by nutritional deficiency of ascorbic acid , formaly occuring in people who had not been to sea results in circular spots , stripes or patches scattered over thighs , arms , and trunk .  एब्सकार्बिक अम्ल कुपोषण की कमी के कारण द्वारा होने वाली बीमारी जो आमतौर से लोगो में होता है जिसके 

In [10]:
print(len(lines_eng),len(lines_hin))

788098 788098


In [0]:
for i in range(len(lines_hin)):
  # splits = line.split('\t')
  #removing extra stuffs 
  eng = lines_eng[i]
  hin = lines_hin[i]
  # remove punctutation and make lower case
  eng = eng.translate(str.maketrans('', '', string.punctuation))
  eng = eng.lower()
  hin = hin.translate(str.maketrans('', '', string.punctuation))
  eng = '<start> ' + eng + ' <end>'
  hin = '<start> ' + hin + ' <end>'
  if len(eng) <= 50:
    eng_sent.append(eng)
  if len(hin) <= 50:
    hin_sent.append(hin)

In [12]:
print(eng_sent[1:10],hin_sent[1:10])
print(len(eng_sent))

['<start> help <end>', '<start> jump <end>', '<start> jump <end>', '<start> jump <end>', '<start> hello <end>', '<start> hello <end>', '<start> cheers <end>', '<start> cheers <end>', '<start> got it <end>'] ['<start> बचाओ <end>', '<start> उछलो <end>', '<start> कूदो <end>', '<start> छलांग <end>', '<start> नमस्ते। <end>', '<start> नमस्कार। <end>', '<start> वाहवाह <end>', '<start> चियर्स <end>', '<start> समझे कि नहीं <end>']
119794


# code to encode dataset and create dictionary using keras tokenizer

In [0]:
def max_length(tensor):
    return max(len(t) for t in tensor)

In [0]:
def tokenize(lang):
    # default filter value is '!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    # an instance of tokenizer
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(
      filters='')
    
    #takes all sentences in lang and makes dictionary for it
    lang_tokenizer.fit_on_texts(lang)
    
    #encodes the text -> (text ->vector)
    tensor = lang_tokenizer.texts_to_sequences(lang)

    #puts appropriate number of zeros after the sent
    # if maxlen of sentences if 100 and any given sentence is of len 20, then
    # it'll pad 80 zeros at the end
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor,
                                                         padding='post')

    # tensor is a vector of n*maxlen
    # and lang_tokenizer is a dictionary mapping word with key 
    return tensor, lang_tokenizer

In [0]:
def load_dataset(inp_lang,targ_lang,load_dataset):
    inp_lang = inp_lang[0:num_examples-1]
    targ_lang = targ_lang[0:num_examples-1]
    input_tensor, inp_lang_tokenizer = tokenize(inp_lang)
    target_tensor, targ_lang_tokenizer = tokenize(targ_lang)

    return input_tensor, target_tensor, inp_lang_tokenizer, targ_lang_tokenizer

# encoding datatset and creating dictionary

In [0]:
# Try experimenting with the size of that dataset
# num_examples = 20000
num_examples = len(eng_sent)
input_tensor, target_tensor, inp_lang, targ_lang = load_dataset(eng_sent,hin_sent,num_examples)

# Calculate max_length of the target tensors
max_length_targ, max_length_inp = max_length(target_tensor), max_length(input_tensor)

In [17]:
print(input_tensor.shape,target_tensor.shape)
print("max length of input and output are : ", max_length_inp,max_length_targ )
print('english dictionary is : ')
print(inp_lang)
print("hindi dictionary is : ")
print(targ_lang)

(119793, 24) (119793, 27)
max length of input and output are :  24 27
english dictionary is : 
hindi dictionary is : 


In [19]:
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=0.3)

# Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

83855 83855 35938 35938


In [0]:
# #This is what our model is going to learn
# def convert(lang, tensor):
#     for t in tensor:
#         if t!=0:
#             print ("%d ----> %s" % (t, lang.index_word[t]))

# print ("Input Language; index to word mapping")
# convert(inp_lang, input_tensor_train[0])
# print ()
# print ("Target Language; index to word mapping")
# convert(targ_lang, target_tensor_train[0])

In [0]:
BUFFER_SIZE = len(input_tensor_train) #training set size
BATCH_SIZE = 32
steps_per_epoch = len(input_tensor_train)//BATCH_SIZE
embedding_dim = 256
units = 1024
vocab_inp_size = len(inp_lang.word_index)+1
vocab_tar_size = len(targ_lang.word_index)+1

#Creates a Dataset whose elements are slices of the given tensors.
dataset = tf.data.Dataset.from_tensor_slices((input_tensor_train, target_tensor_train)).shuffle(BUFFER_SIZE)

#Combines consecutive elements of this dataset into batches.
#eg dataset = tf.data.Dataset.range(8) 
# dataset = dataset.batch(3) 
# list(dataset.as_numpy_iterator()) 
# [ array([0,1,2]), array([3,4,5]) , array([5,6,7])]

dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)

In [21]:
# so dataset loads 64 examples at a time for both input and output
# its a pointer to point to next batch when needed.
print(dataset)


<DatasetV1Adapter shapes: ((32, 24), (32, 27)), types: (tf.int32, tf.int32)>


In [22]:
example_input_batch, example_target_batch = next(iter(dataset))
example_input_batch.shape, example_target_batch.shape

(TensorShape([Dimension(32), Dimension(24)]),
 TensorShape([Dimension(32), Dimension(27)]))

In [23]:
print(example_input_batch)

tf.Tensor(
[[    1  1280   184  8267   127    93     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]
 [    1    17    18     6  7096 26104     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]
 [    1    43    11    80  1486     2     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]
 [    1  7624   388   748     2     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]
 [    1   276  2857    10  9402  2157     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]
 [    1    17    16    11     6 11015  5698  2508     2     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0]
 [    1    13   572    45    96  4006     2     0     0     0     0     0
      0     0     0  

# Making the model


**return_sequences** -> give hidden state for each time step  
**return_state** -> gives (in case of lstm) [hidden state, hidden state, cell state for last time step].  
**both of them** -> (in case of lstm)[hidden state for all time step, hidden state for last time step, cell state for last time step]  
(in case of gru)[hidden state for all time steps, hidden state for final timestep]

In [0]:
class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units, batch_sz):
        super(Encoder, self).__init__()
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.enc_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')

    def call(self, x, hidden):
        #takes input and returns h and y
        x = self.embedding(x)
        output, state ,cell = self.lstm(x, initial_state = hidden)
        encoder_states = [state,cell]
        return output, encoder_states

    def initialize_hidden_state(self):
        state =  tf.zeros((self.batch_sz, self.enc_units))
        return [state,state]

In [25]:
print(vocab_inp_size)
print(embedding_dim,units,BATCH_SIZE)

27488
256 1024 32


In [26]:
# encoder is a class instance having 
encoder = Encoder(vocab_inp_size, embedding_dim, units, BATCH_SIZE)

# not sure how this line is working..
sample_hidden = encoder.initialize_hidden_state()

sample_output, encoder_states = encoder(example_input_batch, sample_hidden)
print ('Encoder output shape: (batch size, sequence length, units) {}'.format(sample_output.shape))
print ('Encoder Hidden state shape: (batch size, units) {}'.format(sample_hidden[0].shape))


Encoder output shape: (batch size, sequence length, units) (32, 24, 1024)
Encoder Hidden state shape: (batch size, units) (32, 1024)


In [27]:

print("shape of sample_hidden",sample_hidden[0].get_shape())

shape of sample_hidden (32, 1024)


# ATTENTION MECHANISAM  CODE


In [0]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)

    def call(self, query, values):
        
        query = query[0]
        # query_with_time_axis shape == (batch_size, 1, hidden size) 
        # we are doing this to broadcast addition along the time axis to calculate the score
        query_with_time_axis = tf.expand_dims(query, 1)

        # score shape == (batch_size, max_length, 1)
        # we get 1 at the last axis because we are applying score to self.V
        # the shape of the tensor before applying self.V is (batch_size, max_length, units)
        score = self.V(tf.nn.tanh(
            self.W1(query_with_time_axis) + self.W2(values)))

        # attention_weights shape == (batch_size, max_length, 1)
        attention_weights = tf.nn.softmax(score, axis=1)

        # context_vector shape after sum == (batch_size, hidden_size)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)

        return context_vector, attention_weights

In [29]:
# this is just testing if attention is working or not
attention_layer = BahdanauAttention(10)
attention_result, attention_weights = attention_layer(sample_hidden, sample_output)

print("Attention result shape: (batch size, units) {}".format(attention_result.shape))
print("Attention weights shape: (batch_size, sequence_length, 1) {}".format(attention_weights.shape))

Attention result shape: (batch size, units) (32, 1024)
Attention weights shape: (batch_size, sequence_length, 1) (32, 24, 1)


In [30]:
print(sample_hidden)

[<tf.Tensor: id=20, shape=(32, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>, <tf.Tensor: id=20, shape=(32, 1024), dtype=float32, numpy=
array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)>]


## DECODER

In [0]:
class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units, batch_sz):
        super(Decoder, self).__init__()
        self.batch_sz = batch_sz
        self.dec_units = dec_units
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_dim)
        self.lstm = tf.keras.layers.LSTM(self.dec_units,
                                       return_sequences=True,
                                       return_state=True,
                                       recurrent_initializer='glorot_uniform')
        self.fc = tf.keras.layers.Dense(vocab_size)

        # used for attention
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        # enc_output shape == (batch_size, max_length, hidden_size)
        # enc_output is the list of all the hidden states
        context_vector, attention_weights = self.attention(hidden, enc_output)

        # x shape after passing through embedding == (batch_size, 1, embedding_dim)
        x = self.embedding(x)

        # x shape after concatenation == (batch_size, 1, embedding_dim + hidden_size)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)

        # passing the concatenated vector to the GRU

        output, hidden, cell = self.lstm(x)
        decoder_states = [hidden,cell]
        # output shape == (batch_size * 1, hidden_size)
        output = tf.reshape(output, (-1, output.shape[2]))

        # output shape == (batch_size, vocab)
        x = self.fc(output)

        return x, decoder_states, attention_weights

### state is hidden state of gru for that time step
### output and state are the same thing here , as this is only for one time step
### output is :  Tensor("decoder_2/gru_5/transpose_1:0", shape=(32, 1, 1024), dtype=float32)
### state is :  Tensor("decoder_2/gru_5/while/Exit_3:0", shape=(32, 1024), dtype=float32)
notice the difference in shapes

### testing if decoder class is working


In [32]:
decoder = Decoder(vocab_tar_size, embedding_dim, units, BATCH_SIZE)

sample_decoder_output, _, _ = decoder(tf.random.uniform((BATCH_SIZE, 1)),
                                      sample_hidden, sample_output)

print ('Decoder output shape: (batch_size, vocab size) {}'.format(sample_decoder_output.shape))

Decoder output shape: (batch_size, vocab size) (32, 36647)


# OPTIMIZER

In [0]:
optimizer = tf.keras.optimizers.Adam()

'''
In the snippet below, there is a single floating point value per example for
`y_true` and `# classes` floating pointing values per example for `y_pred`.
The shape of `y_true` is `[batch_size]` and the shape of `y_pred` is
`[batch_size, num_classes]`.
'''
#Computes the crossentropy loss between the labels and predictions.
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

# only considering cross entropy for incorrectly classified samples.
def loss_function(real, pred):
    # tf.math.logical - Returns the truth value of NOT x element-wise.
    # tf.math.equal - Returns the truth value of (x == y) element-wise.
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    # loss as defined in the image above
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

## Checkpoints (Object-based saving)

In [0]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(optimizer=optimizer,
                                 encoder=encoder,
                                 decoder=decoder)

In [35]:
for (batch, (inp, targ)) in enumerate( dataset.take(steps_per_epoch) ):
    print(targ)
    print(targ.shape)
    break

tf.Tensor(
[[    1   122  6191 19784    17   408     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    1  1035    17  6195     3     2     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    1 18352   980   855   953     2     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    1  2975     5    61     5    15     2     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    1    52   318   348     2     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0     0
      0     0     0]
 [    1 15045  2994 15046     2     0     0     0     0     0     0     0
      0     0     0     0     0     0     0     0     0     0     0   

## Training

1. Pass the *input* through the *encoder* which return *encoder output* and the *encoder hidden state*.
2. The encoder output, encoder hidden state and the decoder input (which is the *start token*) is passed to the decoder.
3. The decoder returns the *predictions* and the *decoder hidden state*.
4. The decoder hidden state is then passed back into the model and the predictions are used to calculate the loss.
5. Use *teacher forcing* to decide the next input to the decoder.
6. *Teacher forcing* is the technique where the *target word* is passed as the *next input* to the decoder.
7. The final step is to calculate the gradients and apply it to the optimizer and backpropagate.

In [0]:
@tf.function
def train_step(inp, targ, enc_hidden):
    '''
    inp is encoded input sentence 
    targ is encoded output sentence(batchsize,len_of_target_word)
    enc_hidden is zero vector of shape (batchsize,units)
    '''
    loss = 0

    with tf.GradientTape() as tape:
        #recoder or tracks variable values
        enc_output, enc_hidden = encoder(inp, enc_hidden)

        dec_hidden = enc_hidden
        
        # targ_lang.word_index['<start>'] - returns dictionary value of start token
        # dec_input is batch_size
        dec_input = tf.expand_dims([targ_lang.word_index['<start>']] * BATCH_SIZE, 1)

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            #enc_output is list of all hidden states of encoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            
            # this is simply the loss between t th predicted word and its prediction
            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            # expand_dims is to change dimention such that it can act as timesteps
            dec_input = tf.expand_dims(targ[:, t], 1)
    
    # explanationn of this part https://www.tensorflow.org/guide/effective_tf2
    batch_loss = (loss / int(targ.shape[1]))

    variables = encoder.trainable_variables + decoder.trainable_variables
    
    gradients = tape.gradient(loss, variables)

    optimizer.apply_gradients(zip(gradients, variables))

    return batch_loss

## operation code 

In [0]:
EPOCHS = 15

for epoch in range(EPOCHS):
    start = time.time()

    # returns a zero vector of shape(batch_size,units)
    enc_hidden = encoder.initialize_hidden_state()
    total_loss = 0

    # dataset is tf.data.Dataset() object.
    for (batch, (inp, targ)) in enumerate( dataset.take(steps_per_epoch) ):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss

    if batch % 100 == 0:
        print('Epoch {} Batch {} Loss {:.4f}'.format(epoch + 1,
                                                   batch,
                                                   batch_loss.numpy()))
    # saving (checkpoint) the model every 2 epochs
    if (epoch + 1) % 2 == 0:
        checkpoint.save(file_prefix = checkpoint_prefix)

    print('Epoch {} Loss {:.4f}'.format(epoch + 1,
                                      total_loss / steps_per_epoch))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## Evaluator
The evaluate function is similar to the training loop, except we don't use teacher forcing here. The input to the decoder at each time step is its previous predictions along with the hidden state and the encoder output.

In [0]:
def preprocess_sentence(eng):
  eng = eng.translate(str.maketrans('', '', string.punctuation))
  eng = eng.lower()
  eng = '<start> ' + eng + ' <end>'
  return eng

In [0]:
def evaluate(sentence):
  # this is for the heat map
  attention_plot = np.zeros((max_length_targ, max_length_inp))

  sentence = preprocess_sentence(sentence)

  #convert input to its embeddings
  inputs = [inp_lang.word_index[i] for i in sentence.split(' ')]
  inputs = tf.keras.preprocessing.sequence.pad_sequences([inputs],
                                                          maxlen=max_length_inp,
                                                          padding='post')
  inputs = tf.convert_to_tensor(inputs)

  # we'll keep appending the predicted word in this
  result = ''

  # got all the hidden states and last encoder hidden state
  hidden = [tf.zeros((1, units)),tf.zeros((1, units))]
  enc_out, enc_hidden = encoder(inputs, hidden)

  # for first timestep decoder hidden state = encoder hidden state
  dec_hidden = enc_hidden
  dec_input = tf.expand_dims([targ_lang.word_index['<start>']], 0)

  # keep feeding words in decoder for no of words in target sentence
  for t in range(max_length_targ):
    predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                          dec_hidden,
                                                          enc_out)

    # storing the attention weights to plot later on
    attention_weights = tf.reshape(attention_weights, (-1, ))
    attention_plot[t] = attention_weights.numpy()

    # max value from probability of all words in target sentence
    predicted_id = tf.argmax(predictions[0]).numpy()
    # print(predictions.shape,predictions)
    result += targ_lang.index_word[predicted_id] + ' '

    if targ_lang.index_word[predicted_id] == '<end>':
      return result, sentence, attention_plot

    # the predicted ID is fed back into the model
    dec_input = tf.expand_dims([predicted_id], 0)

  return result, sentence, attention_plot

In [0]:
# function for plotting the attention weights
def plot_attention(attention, sentence, predicted_sentence):
  fig = plt.figure(figsize=(10,10))
  ax = fig.add_subplot(1, 1, 1)
  ax.matshow(attention, cmap='viridis')

  fontdict = {'fontsize': 14}

  ax.set_xticklabels([''] + sentence, fontdict=fontdict, rotation=90)
  ax.set_yticklabels([''] + predicted_sentence, fontdict=fontdict)

  ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
  ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

  plt.show()

In [0]:
def translate(sentence):
  result, sentence, attention_plot = evaluate(sentence)

  print('Input: %s' % (sentence))
  print('Predicted translation: {}'.format(result))

  attention_plot = attention_plot[:len(result.split(' ')), :len(sentence.split(' '))]
  # plot_attention(attention_plot, sentence.split(' '), result.split(' '))

# Restore the latest checkpoint and test

In [0]:
# restoring the latest checkpoint in checkpoint_dir
checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

## checking

In [0]:
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

In [0]:
translate('I will get back to you')


Input: <start> i will get back to you <end>
Predicted translation: मैं तुम्हारे साथ आऊँगा। <end> 


In [0]:
translate('This cat is nice')


Input: <start> this cat is nice <end>
Predicted translation: यह लिफ़्ट छटी मंज़िल तक ही है। <end> 


In [0]:
translate(u'i will not come')


Input: <start> i will not come <end>
Predicted translation: मैं तुम्हारे साथ आऊँगा। <end> 


In [0]:
translate(u'men are playing')


Input: <start> men are playing <end>
Predicted translation: वे हमारे साथ आएगा। <end> 


In [0]:
translate(u'school is closed')


Input: <start> school is closed <end>
Predicted translation: स्कूल अप्रैल में बोलिए। <end> 


In [0]:
translate(u'why does this happen to me')


Input: <start> why does this happen to me <end>
Predicted translation: तुम क्यों नहीं हो <end> 


In [0]:
translate('this is not working')


Input: <start> this is not working <end>
Predicted translation: यह भी यकीन है। <end> 
